<a href="https://colab.research.google.com/github/shaisk/ML/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import cross_validate

import itertools


import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from IPython.display import display,clear_output
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from ipywidgets import interactive
from ipywidgets import Button, HBox, VBox
import pandas as pd
import numpy as np
import matplotlib.pylab as plt

import os
import signal
import subprocess
import time
from getpass import getpass
import subprocess
from getpass import getpass
from ipywidgets import interact, interact_manual
from ipywidgets import interactive
import ipywidgets as widgets
from IPython.display import display
from IPython.display import display,clear_output
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.layers import Flatten
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 
import pandas as pd
import numpy as np
import string, os 
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cd /content/gdrive/MyDrive/Colab Notebooks


/content/gdrive/MyDrive/Colab Notebooks


In [ ]:
df = pd.read_csv("/content/gdrive/MyDrive/colab/songs.csv")
all_descriptions = list(df.text.values)
corpus = [x for x in all_descriptions]

# Tokenization
t = Tokenizer(num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ', char_level=False, oov_token=None, document_count=0)

def get_sequence_of_tokens(corpus):
    t.fit_on_texts(corpus)
    total_words = len(t.word_index) + 1
    
    input_sequences = []
    for line in corpus:
        token_list = t.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
            
    return input_sequences, total_words
input_sequences, total_words = get_sequence_of_tokens(corpus)
from tensorflow.keras.utils import to_categorical
# pad sequences 
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen = max_sequence_len, padding = 'pre'))
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = to_categorical(label, num_classes = total_words)
    
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(input_sequences)

def create_model(max_sequence_len, total_words):
    model1 = Sequential()
    
    # Add Input Embedding Layer
    model1.add(Embedding(total_words, 10, input_length=max_sequence_len - 1))
    
    # Add Hidden Layer 1 - LSTM Layer
    model1.add(LSTM(50))
    model1.add(Dropout(0.1))

    # model.add(Flatten())
    
    
    # Add Output Layer
    model1.add(Dense(total_words, activation='softmax'))

    model1.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model1

lstm_model = create_model(max_sequence_len, total_words)
lstm_model.fit(predictors, label, epochs=100, verbose=1)




Epoch 1/30
516/516 [==============================] - 113s 184ms/step - loss: 7.0339
Epoch 2/30
516/516 [==============================] - 95s 185ms/step - loss: 6.3258
Epoch 3/30
516/516 [==============================] - 95s 184ms/step - loss: 6.2269
Epoch 4/30
516/516 [==============================] - 94s 182ms/step - loss: 6.1032
Epoch 5/30
516/516 [==============================] - 93s 181ms/step - loss: 5.9672
Epoch 6/30
516/516 [==============================] - 93s 180ms/step - loss: 5.8087
Epoch 7/30
516/516 [==============================] - 93s 180ms/step - loss: 5.6640
Epoch 8/30
516/516 [==============================] - 93s 180ms/step - loss: 5.5204
Epoch 9/30
516/516 [==============================] - 94s 182ms/step - loss: 5.3864
Epoch 10/30
516/516 [==============================] - 93s 181ms/step - loss: 5.2420
Epoch 11/30
516/516 [==============================] - 93s 180ms/step - loss: 5.1272
Epoch 12/30
516/516 [==============================] - 93s 180ms/step - l

In [ ]:
def generate_text(seed_text, next_words, model, max_seq_len):
    for _ in range(next_words):
        token_list = t.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_seq_len-1, padding='pre')
        
        predicted = model.predict_classes(token_list, verbose=0)
        
        output_word = ''
        
        for word,index in t.word_index.items():
            if index == predicted:
                output_word = word
                break
                
        seed_text = seed_text + " " + output_word
        
    return seed_text.title()

In [ ]:
# Create text widget for output
output_text = widgets.Text()

# Create text widget for input
input_text = widgets.Text()

# Define function to bind value of the input to the output variable 
def bind_input_to_output(sender):
    output_text.value = input_text.value

# Tell the text input widget to call bind_input_to_output() on submit
input_text.on_submit(bind_input_to_output)

In [ ]:
input_text.observe(bind_input_to_output)

In [ ]:
# model2 = keras.models.load_model('/content/gdrive/MyDrive/Colab Notebooks/model_lstm/')
# model2 = keras.models.load_model('/content/gdrive/MyDrive/Colab Notebooks/lstm_new/')
# model3 = load_model('/content/my_model.h5')

In [ ]:
max_sequence_len=100

In [ ]:
print(generate_text('Он', 20, lstm_model, 100))

Он Что Что Ты Ты Одна Ты Одна Ты Ты Одна Ты Не Стыдясь Своей Не Стыдясь Своей Не Стыдясь Своей


In [ ]:
!pip3 install transformers

     |████████████████████████████████| 2.3MB 9.1MB/s 
     |████████████████████████████████| 901kB 31.4MB/s 
     |████████████████████████████████| 3.3MB 46.1MB/s 


In [ ]:
from transformers import pipeline, set_seed

In [ ]:
generator = pipeline('text-generation', model='gpt2')

In [ ]:
set_seed(96)


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
def load_tokenizer_and_model(model_name_or_path):
  return GPT2Tokenizer.from_pretrained(model_name_or_path), GPT2LMHeadModel.from_pretrained(model_name_or_path).cuda()


def generate(
    model, tok, text,
    do_sample=True, max_length=100, repetition_penalty=5.0,
    top_k=5, top_p=0.95, temperature=1,
    num_beams=None,
    no_repeat_ngram_size=3
    ):
  input_ids = tok.encode(text, return_tensors="pt").cuda()
  out = model.generate(
      input_ids.cuda(),
      max_length=max_length,
      repetition_penalty=repetition_penalty,
      do_sample=do_sample,
      top_k=top_k, top_p=top_p, temperature=temperature,
      num_beams=num_beams, no_repeat_ngram_size=no_repeat_ngram_size
      )
  return list(map(tok.decode, out))

In [ ]:
tok, model = load_tokenizer_and_model("sberbank-ai/rugpt3small_based_on_gpt2")


In [ ]:
generated = generate(model, tok, "Женя", num_beams=10)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
generated[0]

'Женя, я тебя люблю!\xa0— сказал он и поцеловал ее в губы.\n\nИ она ответила ему тем же:\n\n—\xa0А ты меня любишь?\n\nОн не мог поверить своим ушам. Он никогда еще не был так близок к тому, чтобы сказать ей «нет». Она была для него чем-то таким, что заставляло его чувствовать себя виноватым перед ней за все то, что случилось с ним до того, как он узнал о том, что произошло между ними'

In [65]:
display(input_text)

Text(value='Я ем детей')

In [66]:
btn_lstm = widgets.Button(description='lstm')
btn_gpt_simple = widgets.Button(description='gpt_simple')
btn_gpt3 = widgets.Button(description='gpt3')


output = widgets.Output()
display(output)
def run_lstm(btn_object):
    with output:
        clear_output()
        X = output_text.value
        gen=generate_text(X, 50, lstm_model, 100)
        print(gen)
    

def run_gpt_simple(btn_object):
     with output:
        clear_output()
        gen = generator(output_text.value, max_length=30, num_return_sequences=1)[0]['generated_text']
        print(gen)
    

def run_gpt3(btn_object):
    with output:
        clear_output()
        X = output_text.value
        generated = generate(model, tok, X, num_beams=10)

        print(generated[0])


btn_lstm.on_click(run_lstm)
btn_gpt_simple.on_click(run_gpt_simple)
btn_gpt3.on_click(run_gpt3)




first_line = HBox([btn_lstm, btn_gpt_simple, btn_gpt3])
display(first_line)


Output()